# Amplicon mapping to AGORA2 Genomes
## Notebook 2: Creating the ASV to genome mapping file

#### Kathleen Beilsmith, Argonne National Lab (ANL)
#### ANL Henry Group & UChicago Chang Lab Mouse Gut Microbiome collaboration
#### Data Credit: Megan Kennedy

This notebook takes a compendium of marker gene sequences found in a collection of genomes (from Notebook 1) and compares them to representative sequences from an amplicon dataset.

Inputs:
- JSON file with marker gene compendium
- fasta file with amplicon sequence variant (ASV) sequences (from dada2 or other denoising pipeline)
- forward and reverse primer sequences used to generate amplicons

**ePCR()** uses the marker gene compendium and primer sequences to perform in silico PCR and cut out the region of interest for each copy of the marker gene (eAmplicon).

**fuzzy_match_amplicons_to_genomes()** compares each ASV from the dataset to the set of eAmplicons generated with the primers and returns matches above a specified sequence identity threshold. The result is a JSON mapping file with one entry per ASV with a list of names for genomes containing matching eAmplicons.

Sequence alignment scores are generated with Biopython Align.PairwiseAligner() with 'global' mode. The match and mismatch scores are 1 and 0, respectively, and the penalties for opening and extending gaps are -1 and -0.5, respectively. 
https://biopython.org/docs/1.75/api/Bio.Align.html#Bio.Align.PairwiseAlignment
With these settings, the rounded **alignment score** corresponds to the number of nucleotides matching in the sequences. 

The **expected** number of nucleotide matches at the given identity threshold is calculated as the identity threshold (float) x amplicon sequence length (int), rounded.

If the alignment score is greater than or equal to the expected score, the amplicon from the dataset is considered matched to an eAmplicon from the compendium (AGORA2) at the identity threshold.

**best_match_amplicons_to_genomes()** uses fuzzy_match_amplicons_to_genomes() to get results at every identity threshold between 1 and a provided lower bound. The result is a JSON mapping file with one entry per ASV with a list of names for genomes containing matching eAmplicons. Only the results for the highest similarity level at which a match was found are stored.

In [2]:
############################
# Required modules
############################

import numpy as np

import os
import modelseedpy
import json as _json
from modelseedpy import RPCClient
from Bio import SeqUtils
from Bio import SeqIO
from Bio.Seq import Seq

import json
import glob

from Bio.Align import MultipleSeqAlignment
from Bio import Align
from Bio import pairwise2 

### Use compendium of marker gene copies and primer sequences to do ePCR.

In [2]:
with open('/Users/kbeilsmith/Desktop/2024_AmtG/AmtG_0.0.2_test_files/test_Compendium.json') as json_file:
    rRNA_dict = json.load(json_file)

In [3]:
############################
# Primer sequences
############################

f_pr = Seq('AYTGGGYDTAAAGNG')
r_pr = Seq('CCGTCAATTYHTTTRAGT')

In [4]:
# Given a marker gene compendium and primers
# this function returns an eAmplicon sequence for each marker gene copy,
# with the information about the location (genome) of the marker gene copy.

def ePCR(f_primer,r_primer,marker_gene_compendium):
    
    eAmplicon_set = {}
    
    # For each marker gene copy in the compendium:
    for feature in marker_gene_compendium:
    # print(rRNA_dict[feature])
    # print(feature)
        
        # Find exact sequence matches with the F and then R primers.
        fwd_ePCR = SeqUtils.nt_search(str(marker_gene_compendium[feature]),f_primer)
        rev_ePCR = SeqUtils.nt_search(str(marker_gene_compendium[feature]),r_primer.reverse_complement())
        
        # If both primers match:
        if len(fwd_ePCR) >= 2 and len(rev_ePCR) >=2:
            print(f"Marker gene copy: {feature}")
            print(f"Forward primer match: {fwd_ePCR}")
            print(f"Reverse primer match: {rev_ePCR}")
            # Add primer length to find start of amplicon sequence from forward match.
            start_cut = fwd_ePCR[1]+len(f_primer)
            # The beginning of the reverse primer match is the end of the amplicon.
            end_cut = rev_ePCR[1]
            # Use the start and end positions to get the eAmplicon sequence.
            full_feature_seq = marker_gene_compendium[feature]
            eAmplicon = full_feature_seq[start_cut:end_cut]
            eAmplicon_length = len(eAmplicon)
            print(f"eAmplicon length: {eAmplicon_length}")
            print(f"eAmplicon sequence: {eAmplicon} \n")
        
        # Store the eAmplicon with the marker gene record of genome, contig, direction, etc.
        eAmplicon_set[feature] = eAmplicon
        
    return eAmplicon_set

In [5]:
mk_dict = ePCR(f_pr,r_pr,rRNA_dict)

Marker gene copy: GCA_000210035.1_ASM21003v1_genomic.fna;FP929055.1;+;147;1539
Forward primer match: ['A[CT]TGGG[CT][AGT]TAAAG[GATC]G', 555]
Reverse primer match: ['ACT[CT]AAA[AGT][AG]AATTGACGG', 901]
eAmplicon length: 331
eAmplicon sequence: AGCGTAGACGGAGTGGCAAGTCTGATGTGAAAACCCGGGGCTCAACCCCGGGACTGCATTGGAAACTGTCAATCTGGAGTACCGGAGAGGTAAGCGGAATTCCTAGTGTAGCGGTGAAATGCGTAGATATTAGGAGGAACACCAGTGGCGAAGGCGGCTTACTGGACGGTAACTGACGTTGAGGCTCGAAAGCGTGGGGAGCAAACAGGATTAGATACCCTGGTAGTCCACGCCGTAAACGATGACTACTAGGTGTCGGGCAGCAAAGCTGTTCGGTGCCGCAGCAAACGCAATAAGTAGTCCACCTGGGGAGTACGTTCGCAAGAATGAA 

Marker gene copy: GCA_000025225.2_ASM2522v1_genomic.fna;CP001850.2;+;255524;1534
Forward primer match: ['A[CT]TGGG[CT][AGT]TAAAG[GATC]G', 551]
Reverse primer match: ['ACT[CT]AAA[AGT][AG]AATTGACGG', 898]
eAmplicon length: 332
eAmplicon sequence: CGTGCAGGCGGGCTGATAAGTCAGATGTGAAATCCCCGAGCTTAACTCGGGAACTGCATCTGATACTGTTGGTCTTGAGTGCTGGAGAGGATAGTGGAATTCCTAGTGTAGCGGTAAAATGCGCAGATATTAGGAGGAACACCAGTGGCGAAGGCGGCTATCTGGACAGTAACTGACG

### Use eAmplicons and dataset representative sequences fasta to find matches between amplicons and genomes in the collection.

In [6]:
mk_seqs = "/Users/kbeilsmith/Desktop/2024_AmtG/AmtG_0.0.2_test_files/rep_seqs.fasta"

In [7]:
count = 0
for rep_seq in SeqIO.parse(mk_seqs, "fasta"):
    count = count+1
print(f"Amplicons in dataset: {count} \n")

Amplicons in dataset: 10 



##### Set parameters for pairwise alignment

mode: 'global'

match_score = 1

mismatch_score = 0

open_gap_score = -1

extend_gap_score = -0.5

In [8]:
aligner = Align.PairwiseAligner(mode='global', match_score=1, mismatch_score=0, open_gap_score = -1, extend_gap_score = -0.5)

In [9]:
# Given a file of amplicon sequences and a set of eAmplicons generated from from genomes,
# this function returns a record of the marker gene copies matching each amplicon,
# which includes genome name and location.

def fuzzy_match_amplicons_to_genomes(rep_seqs,eAmplicon_set,identity_threshold):
    
    percent_identical = identity_threshold
    # print(f"Identity threshold {percent_identical}. \n")
    percent_mismatch = 1 - percent_identical
    
    AmtG_dict = {}
    
    # For each amplicon from the dataset:
    for amplicon in SeqIO.parse(rep_seqs, "fasta"):
        amplicon_id = f"{amplicon.id}"
        print(f"Amplicon: {amplicon.id}")
        
        AmtG_list = []
        # For each eAmplicon from the genomes:
        for eAmplicon in eAmplicon_set:
            
            # Check for undefined nucleotides.
            if eAmplicon_set[eAmplicon].find("N")< 0:
                
                # Find the number of expected matching nucleotides for this threshold.
                amplicon_length = len(eAmplicon_set[eAmplicon])
                expected_matches = percent_identical*amplicon_length
                expected_matches_round = round(expected_matches)
                # print(f"Expected matches = {expected_matches_round} of {amplicon_length} nucleotides.")
                
                # Generate a scored alignment.
                alignment_score = aligner.score(amplicon.seq, eAmplicon_set[eAmplicon])
                alignment_score_round = round(alignment_score)
                #print(f"Alignment score = {alignment_score_round}.")
                
                # Compare the alignment score and expected matches.
                if alignment_score_round >= expected_matches_round:
                    # print(f"MATCH: alignment score {alignment_score_round} exceeds expected score {expected_matches_round}.")
                    # Add matching 16S copies to list.
                    AmtG_list.append(eAmplicon)
            
        match_count = len(AmtG_list)
        print(f"Matches: {match_count}")           
                #else:
                    #print(f"NO MATCH: alignment score {alignment_score_round} below expected score {expected_matches_round}.")
            #else:
                #print(f"Reference amplicon has undefined nucleotides.")
                
        # Create entry with amplicon ID and matching genomes list.
        AmtG_dict[amplicon_id] = AmtG_list

    return AmtG_dict

In [10]:
mk_matches = fuzzy_match_amplicons_to_genomes(mk_seqs,mk_dict,.95)

Amplicon: 001cfeeb86f51bd8ae2b79268c8d806d
Matches: 0
Amplicon: 003902c5913081651d13fa0042ecac85
Matches: 0
Amplicon: 003ef22a3dcf8c6e655f6fcb9e5b8883
Matches: 0
Amplicon: 004d919d8ff500a00bfe768062d3e30a
Matches: 0
Amplicon: 006acab58c360970518d08544e77f20f
Matches: 0
Amplicon: 0073d25591f05f6cfc230a89f51cd841
Matches: 0
Amplicon: 007a974b0637bc17763c30e89fbe0fec
Matches: 0
Amplicon: 008fb713223dbad791ce1c39c7d964b3
Matches: 0
Amplicon: 00a930c5686229958d943fd2a962c3ef
Matches: 0
Amplicon: 00bd2dd899758eea2b046d677293459a
Matches: 2


In [11]:
mk_matches

{'001cfeeb86f51bd8ae2b79268c8d806d': [],
 '003902c5913081651d13fa0042ecac85': [],
 '003ef22a3dcf8c6e655f6fcb9e5b8883': [],
 '004d919d8ff500a00bfe768062d3e30a': [],
 '006acab58c360970518d08544e77f20f': [],
 '0073d25591f05f6cfc230a89f51cd841': [],
 '007a974b0637bc17763c30e89fbe0fec': [],
 '008fb713223dbad791ce1c39c7d964b3': [],
 '00a930c5686229958d943fd2a962c3ef': [],
 '00bd2dd899758eea2b046d677293459a': ['GCF_000157535.1_ASM15753v1_genomic.fna;NZ_GG688487.1;-;2137;1573',
  'GCF_000157555.1_ASM15755v1_genomic.fna;NZ_GG688445.1;+;133;1573']}

In [13]:
# Given a file of amplicon sequences and a set of eAmplicons generated from from genomes,
# this function returns a record of the marker gene copies matching each amplicon,
# which includes genome name and location.

def best_match_amplicons_to_genomes(rep_seqs,eAmplicon_set,identity_threshold):
    
    percent_identical = identity_threshold
    print(f"Finding matches with at least {percent_identical} identity. \n")   
    identity_range = np.flip(np.arange(percent_identical, 1.01, 0.01).round(2))
    
    AmtG_best_matches = {}
    
    for i in identity_range:
        print(" ")
        print(f"Running identity threshold {i}.")
            
        matches = fuzzy_match_amplicons_to_genomes(rep_seqs,eAmplicon_set,i)
        matched_ASVs = {ASV:hits for (ASV,hits) in matches.items() if len(hits) > 0}
        matched_ASV_names = list(matched_ASVs.keys())
        #print(f"Matched ASVs: {matched_ASV_names}.")
    
        for (ASV,hits) in matches.items():
            # print(ASV)
            if ASV not in AmtG_best_matches.keys():
                # print(hits)
                if len(hits) > 0:
                    print(f"Adding hits for {ASV}.")
                    # print(ASV)
                    # print(hits)
                    AmtG_best_matches[ASV] = [i,hits]
            elif ASV in AmtG_best_matches.keys():
                print(f"{ASV} already has a best hit.")

    return AmtG_best_matches

In [14]:
best_matches = best_match_amplicons_to_genomes(mk_seqs,mk_dict,.90)

Finding matches with at least 0.9 identity. 

 
Running identity threshold 1.0.
Amplicon: 001cfeeb86f51bd8ae2b79268c8d806d
Matches: 0
Amplicon: 003902c5913081651d13fa0042ecac85
Matches: 0
Amplicon: 003ef22a3dcf8c6e655f6fcb9e5b8883
Matches: 0
Amplicon: 004d919d8ff500a00bfe768062d3e30a
Matches: 0
Amplicon: 006acab58c360970518d08544e77f20f
Matches: 0
Amplicon: 0073d25591f05f6cfc230a89f51cd841
Matches: 0
Amplicon: 007a974b0637bc17763c30e89fbe0fec
Matches: 0
Amplicon: 008fb713223dbad791ce1c39c7d964b3
Matches: 0
Amplicon: 00a930c5686229958d943fd2a962c3ef
Matches: 0
Amplicon: 00bd2dd899758eea2b046d677293459a
Matches: 0
 
Running identity threshold 0.99.
Amplicon: 001cfeeb86f51bd8ae2b79268c8d806d
Matches: 0
Amplicon: 003902c5913081651d13fa0042ecac85
Matches: 0
Amplicon: 003ef22a3dcf8c6e655f6fcb9e5b8883
Matches: 0
Amplicon: 004d919d8ff500a00bfe768062d3e30a
Matches: 0
Amplicon: 006acab58c360970518d08544e77f20f
Matches: 0
Amplicon: 0073d25591f05f6cfc230a89f51cd841
Matches: 0
Amplicon: 007a974b063

In [15]:
best_matches

{'00bd2dd899758eea2b046d677293459a': [0.95,
  ['GCF_000157535.1_ASM15753v1_genomic.fna;NZ_GG688487.1;-;2137;1573',
   'GCF_000157555.1_ASM15755v1_genomic.fna;NZ_GG688445.1;+;133;1573']],
 '007a974b0637bc17763c30e89fbe0fec': [0.92,
  ['GCA_000210035.1_ASM21003v1_genomic.fna;FP929055.1;+;147;1539']],
 '004d919d8ff500a00bfe768062d3e30a': [0.9,
  ['GCA_000210035.1_ASM21003v1_genomic.fna;FP929055.1;+;147;1539']]}

### Write out mappings JSON file. This file is used as input for the community modeling pipeline.

In [16]:
# Reformat to store only threshold and unique genome names for the matches.

out_best_matches = {}

for match in best_matches:
    # print(best_matches[match][1])
    genome_list = []
    for entry in best_matches[match][1]:
        #print(entry.split("_")[0:2])
        #print(entry.split("_")[0:2])
        entry_name = entry.split("_")[0:2]
        # print("_".join(entry_name))
        entry_name = "_".join(entry_name)
        genome_list.append(entry_name)
    out_best_matches[match] = [best_matches[match][0], list(set(genome_list))]
    
print(out_best_matches)

{'00bd2dd899758eea2b046d677293459a': [0.95, ['GCF_000157535.1', 'GCF_000157555.1']], '007a974b0637bc17763c30e89fbe0fec': [0.92, ['GCA_000210035.1']], '004d919d8ff500a00bfe768062d3e30a': [0.9, ['GCA_000210035.1']]}


In [17]:
# Write out

output_json = "/Users/kbeilsmith/Desktop/2024_AmtG/AmtG_0.0.2_test_files/test_best_mappings.json"
with open(output_json, "w") as outfile:
    json.dump(out_best_matches, outfile)